# Tutorial 1: using a Quantum Device to solve MIS


The high-level goal of this tutorial is to execute an instance of the classic NP-hard Maximum Independent Set (MIS) problem, using both classical and quantum methods.

By the end of this notebook, you will know how to:

- Setup import for standard MIS benchmarking DIMACS datasets.
- Setup compilation and execution of these graphs for execution on both Classical and Quantum Device (either an emulator or a physical QPU).
- Launch the execution and extract relevant results.

## Dataset preparation

As in any MIS, we first need to load and prepare data. MIS can work with many types of graphs. For this tutorial, we will use the standard [DIMACS datasets](https://oeis.org/A265032/a265032.html) to benchmark the process.

In [1]:
# Ignore warnings for this tutorial.
import logging
import os
import sys

logger = logging.getLogger()
logger.disabled = True

sys.stderr = open(os.devnull, 'w')

In [2]:
import networkx as nx

# Create a new networkx graph instance to be populated with DIMACS data.
graph = nx.Graph()


with open("./datasets/a265032_1tc.32.txt", "r") as f:
    for line in f:
        if line.startswith("c"):  # Comment line in DIMACS file.
            continue
        elif line.startswith("p"):  # Problem definition, i.e. # nodes and edges.
            _, _, num_nodes, num_edges = line.strip().split()
            # Preset graph node labels as there might be isolated ones.
            graph.add_nodes_from(range(1, int(num_nodes) + 1))

        elif line.startswith("e"):
            _, node1, node2 = line.strip().split()
            graph.add_edge(int(node1), int(node2))

# Let's check what the graph looks like.
print(graph)


p edge         32        68

e         2         3

e         2         5

e         3         5

e         3         9

e         4         6

e         4         7

e         4        10

e         5         9

e         5        17

e         6         7

e         6        10

e         6        11

e         6        18

e         7        10

e         7        11

e         7        13

e         7        19

e         8        12

e         8        14

e         8        20

e         9        17

e        10        11

e        10        13

e        10        18

e        10        19

e        11        13

e        11        18

e        11        19

e        11        21

e        12        14

e        12        15

e        12        20

e        12        22

e        13        19

e        13        21

e        13        25

e        14        15

e        14        20

e        14        22

e        14        23

e        14        26

e        15        22

e    

## Solving the MIS using the classical CPLEX solver

Let's first solve using the MIS features and IBM's standard classical solver [Cplex](https://www.ibm.com/products/ilog-cplex-optimization-studio). 

In [3]:
from mis.solver.solver import MISInstance
from mis.pipeline.config import SolverConfig
from mis import MISSolver
# Define classical solver configuration
from mis.shared.types import MethodType

# Set solver config to run Cplex.
config = SolverConfig(
    method = MethodType.EAGER,
    max_iterations=1
)

# Create the MIS instance
instance = MISInstance(graph)

# Run the solver and retrieve results.
solver = MISSolver(instance, config)
solutions = solver.solve().result()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution cost:", solutions[0].energy)

MIS solution: [32, 1, 2, 4, 8, 9, 15, 16, 18, 25, 26, 30]
Solution cost: 0


## Solving using the quantum SDK QuTiP

In [4]:
from mis.pipeline.backends import QutipBackend


config = SolverConfig(
    method = MethodType.EAGER,
    backend = QutipBackend(),
    max_iterations=1
)

# Create the MIS instance
instance = MISInstance(graph)


# Run the solver
solver = MISSolver(instance, config)
solutions = solver.solve().result()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution cost:", solutions[0].energy)

MIS solution: [32, 1, 2, 4, 8, 9, 15, 16, 18, 25, 26, 30]
Solution cost: 0


## Solving using Remote QPU backend 

In [5]:
from mis.pipeline.backends import RemoteQPUBackend

USERNAME="username"
PROJECT_ID="123"
PASSWORD=None

if PASSWORD is not None:
    config = SolverConfig(
        method = MethodType.EAGER,
        backend = RemoteQPUBackend(
            username=USERNAME,
            project_id=PROJECT_ID,
            password=PASSWORD
        ),
        max_iterations=1
    )

    # Create the MIS instance
    instance = MISInstance(graph)

    # Run the solver
    solver = MISSolver(instance, config)
    solutions = solver.solve().result()

    # Display results
    print("MIS solution:", solutions[0].nodes)
    print("Solution cost:", solutions[0].energy)